In [29]:
#Instalación de paquetes necesarios
!pip install gradio

#Importamos las librerias necesarias
import pandas as pd
import zipfile
import os
import numpy as np
import gradio as gr


#asociación de Collab con Kagle
  #carga de archivo json
from google.colab import files
#la siguiente linea la comento una vez cargado el token
#files.upload()

  #creamos una carpeta oculta en el sistema para alojar el json y le damos permisos
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#indicamos el dataset que queremos usar
!kaggle datasets download -d nehalbirla/vehicle-dataset-from-cardekho

# Descomprimir el archivo ZIP
with zipfile.ZipFile('vehicle-dataset-from-cardekho.zip', 'r') as zip_ref:
    zip_ref.extractall('cardekho_data')

# Verifica los archivos extraídos
os.listdir('cardekho_data')

# Cargar el dataset
df = pd.read_csv("cardekho_data/CAR DETAILS FROM CAR DEKHO.csv")


#---Preparación del dato---
#1º Añadir columnas adicionales (de forma aleatoria)
#colores de los vehículos
colores = ['rojo', 'azul', 'negro', 'blanco', 'gris', 'verde']
df['color'] = np.random.choice(colores, size=len(df))

#¿Incluye garantía?
df['garantia'] = np.random.choice(['sí', 'no'], size=len(df))

#Número de puertas
df['numero_puertas'] = np.random.choice([2, 3, 5], size=len(df))

#¿Incluye rueda de respuesto?
df['rueda_repuesto'] = np.random.choice(['sí', 'no'], size=len(df))

#¿Equipo de sonido compatible con Android Auto o Apple Car?
df['compatible_apple_android'] = np.random.choice(['sí', 'no'], size=len(df))

#2º Conversión del precio (de Rupias Indias a Euros)
# Tipo de cambio aproximado: 1 EUR = 90 INR
tipo_cambio = 90

#3º Cambiar nombre de valor "Petrol" por "Gasolina"
df['fuel'] = df['fuel'].replace('Petrol', 'Gasolina')

#4º Crear nueva columna con el precio en euros
df['precio_euros'] = (df['selling_price'] / tipo_cambio).round(2)

#5º Quedarme con las columnas que me interesan y ocultar las irrelevantes"df.head"
df[['name', 'precio_euros', 'year', 'seller_type', 'color', 'km_driven', 'fuel', 'transmission', 'owner', 'garantia', 'numero_puertas', 'compatible_apple_android', 'rueda_repuesto', ]].head()
df.drop(columns=['selling_price'], inplace=True)

#6º Limpiar del dataset de valores nulos
df.isnull().sum()

#---FIN de Preparación del dato---

# Almacenar mi propio CSV limpio sin la columna de indexación que crea Pandas
df.to_csv("archivolimpio.csv", index=False)

# Mostrar las primeras filas para verificar
#df.head()

#---Creación de interfaz con Gradio---
# Definir la función de las recomendaciones
def recomendar_vehiculos(tipo_combustible, kilometraje_maximo, precio_maximo, anio_maximo):
    # Filtrar el DataFrame según las preferencias del usuario
    recomendados = df[(df['fuel'] == tipo_combustible) &
                       (df['km_driven'] <= kilometraje_maximo) &
                       (df['precio_eur'] <= precio_maximo) &
                       (df['year'] >= anio_maximo)]

    # Devolver las recomendaciones en formato de tabla
    return recomendados[['name', 'year', 'fuel', 'km_driven', 'precio_eur']]

#Creación de la interfaz interactiva
interface = gr.Interface(
    fn=recomendar_vehiculos,  # Función que ejecutará el código para mostrar recomendaciones
    inputs=[
        gr.Dropdown(choices=df['fuel'].unique().tolist(), label="Tipo de Combustible"),
        gr.Slider(minimum=0, maximum=500000, value=100000, label="Kilometraje Máximo"),
        gr.Slider(minimum=0, maximum=50000, value=20000, label="Precio Máximo en EUR"),
        gr.Slider(minimum=1990, maximum=2025, value=2020, label="Año Máximo de Fabricación")
    ],
    outputs="dataframe"  # Mostrar los resultados en formato tabla
)

# Ejecución de la interfaz
interface.launch()




Dataset URL: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho
License(s): DbCL-1.0
vehicle-dataset-from-cardekho.zip: Skipping, found more recently modified local copy (use --force to force download)
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9aa842847f64922aae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
